<a href="https://colab.research.google.com/github/DianaDaInLee/PredEthnorace/blob/main/PredEthnorace_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install duckduckgo_search
#pip install DeepFace
#pip install pyreadr

In [19]:
import sys
import os
import requests
import random
import re
import json
import time
import logging
import urllib
import scipy.misc
import cv2
import numpy as np
import glob
import socket
import pandas as pd
import pickle
import shutil
import tensorflow as tf
import keras
from matplotlib import pyplot
#from duckduckgo_search import ddg_images
from deepface import DeepFace
from PIL import Image
from mtcnn.mtcnn import MTCNN
from socket import timeout
from matplotlib.pyplot import imread
socket.setdefaulttimeout(90)

In [5]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Racial Prediction/Guidelines')
os.getcwd()

Mounted at /content/drive


'/content/drive/My Drive/Racial Prediction/Guidelines'

In [40]:
def img_check(imgfolder, detector = 'opencv', distance = 'euclidean_l2', model = 'OpenFace'):
    img_list = glob.glob(imgfolder + "/*jpeg") + glob.glob(imgfolder + "/*jpg") + glob.glob(imgfolder + "/*png")

    # Check how many faces there are
    df_nface = pd.DataFrame(columns=['image', 'n_faces'])
    for img in img_list:
        faces = MTCNN().detect_faces(pyplot.imread(img))
        df_nface = df_nface.append({'image': img, 'n_faces': len(faces)}, ignore_index = True)

    # Check for duplicates
    unique_pairs = []
    for i in range(len(img_list)):
        for j in range(i + 1, len(img_list)):
            # Create a tuple representing a unique pair of elements
            pair = (img_list[i], img_list[j])
            unique_pairs.append(pair)

    df_dupes = pd.DataFrame(columns=['image1', 'image2', 'dupe', 'distance', 'model', 'detector'])
    for img in unique_pairs:
        pairs_chk = DeepFace.verify(img1_path = img[0],
                                    img2_path = img[1],
                                    enforce_detection = False,
                                    model_name = model,
                                    distance_metric = distance,
                                    detector_backend= detector)
        df_dupes = df_dupes.append({'image1': img[0],
                                    'image2': img[1],
                                    'dupe': pairs_chk['verified'],
                                    'distance': pairs_chk['distance'],
                                    'model': pairs_chk['model'],
                                    'detector': pairs_chk['detector_backend']}, ignore_index = True)

    return df_nface, df_dupes


In [47]:
def img_pred(imgfolder, detector = "opencv", out_csv = True):
    img_list = glob.glob(imgfolder + "/*jpeg") + glob.glob(imgfolder + "/*jpg") + glob.glob(imgfolder + "/*png")

    if len(img_list)==0:
        print('Error: No images exist in folder' + imgfolder)

    else:
        all = []
        for im in img_list:
            prd = DeepFace.analyze(im, ['race'], detector_backend = detector, enforce_detection= False)
            df  = pd.DataFrame(prd).T
            res = pd.concat([df.iloc[0,:].apply(pd.Series), df.iloc[1,:]], axis = 1)
            res['fn'] = im
            res['detector'] = detector
            all.append(res)

        all = pd.concat(all)
        if (out_csv == True):
            all.to_csv(imgfolder + "/img_pred.csv", index = False)

        return(all)

In [42]:
a, b = img_check(imgfolder = 'demo')

1/1 [==============================] - 0s 189ms/step


In [43]:
a

,image,n_faces
0,demo/image2.jpeg,1
1,demo/image1.jpeg,47
2,demo/image4.jpeg,1
3,demo/image3.jpeg,1
4,demo/image5.jpg,1


image1 is detected to have 47 individual faces. You might want to consider images like this to avoid inaccurate racial prediction.

In [44]:
b

,image1,image2,dupe,distance,model,detector
0,demo/image2.jpeg,demo/image1.jpeg,False,0.553669,OpenFace,opencv
1,demo/image2.jpeg,demo/image4.jpeg,False,0.991721,OpenFace,opencv
2,demo/image2.jpeg,demo/image3.jpeg,False,1.027392,OpenFace,opencv
3,demo/image2.jpeg,demo/image5.jpg,False,0.978743,OpenFace,opencv
4,demo/image1.jpeg,demo/image4.jpeg,True,0.525695,OpenFace,opencv
5,demo/image1.jpeg,demo/image3.jpeg,False,0.778902,OpenFace,opencv
6,demo/image1.jpeg,demo/image5.jpg,True,0.479172,OpenFace,opencv
7,demo/image4.jpeg,demo/image3.jpeg,False,1.132091,OpenFace,opencv
8,demo/image4.jpeg,demo/image5.jpg,True,0.409905,OpenFace,opencv
9,demo/image3.jpeg,demo/image5.jpg,False,1.141893,OpenFace,opencv


image4 and image5 are detected as a duplicate. You might want to remove one of them if your objective is to keep unique faces only.

Generate predictions

In [48]:
img_pred(imgfolder = 'demo', detector = "opencv", out_csv = True)

Action: race: 100%|██████████| 1/1 [00:00<00:00,  1.49it/s]


,asian,indian,black,white,middle eastern,latino hispanic,dominant_race,fn,detector
0,7.306429e-01,7.287341e-01,9.475515e+01,0.157305,0.160390,3.467783,black,demo/image2.jpeg,opencv
1,7.049768e+00,1.428247e+00,5.642956e-01,61.435980,15.889704,13.632008,white,demo/image2.jpeg,opencv
0,1.311560e+01,1.393238e+01,2.471523e+01,10.157196,7.962737,30.116857,latino hispanic,demo/image1.jpeg,opencv
1,5.734332e+00,3.885593e+00,2.028400e+00,42.733133,18.242975,27.375564,white,demo/image1.jpeg,opencv
2,6.678910e+01,4.210242e+00,2.645902e+01,0.151518,0.027926,2.362195,asian,demo/image1.jpeg,opencv
3,1.325321e+00,9.619726e-01,9.824114e-02,61.161929,20.932490,15.520045,white,demo/image1.jpeg,opencv
4,3.822673e-01,1.792150e+00,9.418728e-02,38.126113,43.464130,16.141153,middle eastern,demo/image1.jpeg,opencv
5,1.908147e+01,1.244506e+00,2.069888e-01,48.316136,8.532615,22.618293,white,demo/image1.jpeg,opencv
6,3.825514e-03,9.176248e-03,1.701483e-04,93.598771,2.442959,3.945101,white,demo/image1.jpeg,opencv
7,5.155594e+01,7.452389e+00,4.043555e+00,3.339821,1.521880,32.086423,asian,demo/image1.jpeg,opencv
